In [ ]:
# Imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai

In [ ]:
# Load environment variables
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

# Debugging: Check API key availability
print(f"OpenAI API Key: {'Exists' if openai_api_key else 'Not set'}")
print(f"Google API Key: {'Exists' if google_api_key else 'Not set'}")

In [ ]:
# Initialize OpenAI and Google Generative AI
openai = OpenAI()
google.generativeai.configure()

# Model configurations
gpt_model = "gpt-4o-mini"
google_model = "gemini-2.0-flash-exp"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

google_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

# Initialize message history
gpt_messages = ["Hi there"]
google_messages = ["Hi"]

In [ ]:
# Model functions

def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, google in zip(gpt_messages, google_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": google})
    
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

def call_google():
    messages = []
    for gpt, google_message in zip(gpt_messages, google_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": google_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})

    model = google.generativeai.GenerativeModel(
        model_name=google_model,
        system_instruction=google_system
    )
    
    response = model.generate_content([m["content"] for m in messages])
    return response.text

In [ ]:
# Conversation loop
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Google:\n{google_messages[0]}\n")

for _ in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    google_next = call_google()
    print(f"Google:\n{google_next}\n")
    google_messages.append(google_next)